In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')
small_llm = ChatOpenAI(model='gpt-4o-mini')

In [ ]:
import mysql.connector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="noticeboard"
)
cursor = conn.cursor(dictionary=True)

# 1. 데이터 조회
cursor.execute("""
    SELECT page_id, domain, page_url, description
    FROM page_metadata
""")
rows = cursor.fetchall()

# 2. LangChain 데이터 형식(Document)으로 변환
documents = []
for row in rows:
    # 벡터화할 핵심 텍스트: description
    page_content = row['description'] 
    
    # 저장할 메타데이터: 검색 결과에서 꺼내 쓸 정보들
    metadata = {
        "page_id": row["page_id"],
        "domain": row["domain"],
        "page_url": row["page_url"]
    }
    
    documents.append(Document(page_content=page_content, metadata=metadata))

# 3. Embedding 모델 설정 및 Vector DB 저장
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Chroma DB에 저장
vector_db = Chroma.from_documents(
    documents, 
    embeddings, 
    persist_directory="./my_vector_db"
)

print("데이터 적재 완료!")

데이터 적재 완료!


In [9]:
# 사용자의 질문(의도) 시뮬레이션
query = "비밀번호를 잊어버렸는데 로그인 어떻게 해?"

# 유사도 검색 수행 (가장 유사한 2개 가져오기)
search_results = vector_db.similarity_search(query, k=2)

for doc in search_results:
    print(f"도메인: {doc.metadata['domain']}")
    print(f"경로(URL): {doc.metadata['page_url']}")
    print(f"매칭된 설명: {doc.page_content}")
    print("-" * 30)

도메인: 회원
경로(URL): /login
매칭된 설명: 사용자가 아이디와 비밀번호로 로그인하는 화면이다. 인증 성공 시 서비스에 접근할 수 있다.
------------------------------
도메인: 보안
경로(URL): /security/2fa
매칭된 설명: 사용자의 계정 보안을 강화하기 위해 2단계 인증을 설정하는 화면이다. OTP 또는 QR 기반 인증을 설정한다.
------------------------------
